23122040 Nguyễn Văn Linh

23122022 Trần Hoàng Gia Bảo

23122026 Trần Chấn Hiệp

23122040 Nguyễn Thị Mỹ Kim

Bài làm chạy trong colab notebook

In [20]:
!pip install spacy pandas numpy matplotlib seaborn nltk

In [21]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import math
from typing import Dict, List, Tuple
import csv
import spacy
import pandas as pd
import re
import string
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [22]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
class TextPreprocessor:
    def __init__(self):
        # Khởi tạo lemmatizer
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def clean_text(self, text):
        """
        text: một đoạn text (cụ thể ở đây là text kết hợp của subject và messasge)
        Làm sạch text - loại bỏ non-words
        """
        if pd.isna(text) or text == '':
            return ''

        # Chuyển về chữ thường
        text = text.lower()

        # Loại bỏ email addresses
        text = re.sub(r'\S+@\S+', '', text)

        # Loại bỏ URLs
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

        # Loại bỏ HTML tags
        text = re.sub(r'<[^>]+>', '', text)

        # Loại bỏ số (numbers)
        text = re.sub(r'\d+', '', text)

        # Loại bỏ dấu câu và ký tự đặc biệt, chỉ giữ lại chữ cái
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)

        # Loại bỏ ký tự tabs và xuống dòng
        text = re.sub(r'[\t\n\r\f\v]', ' ', text)

        # Loại bỏ khoảng trắng thừa
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def tokenize_and_lemmatize(self, text):
        """
        text: một đoạn text (cụ thể ở đây là text kết hợp của subject và messasge)
        Tách từ, loại bỏ stop words và thực hiện lemmatization đưa từ về định dạng gốc
        """
        if not text:
            return []

        words = word_tokenize(text)

        processed_words = []
        for word in words:
            if (word not in self.stop_words and # nếu từ không phải stop_word
                len(word) >= 3 and  # lấy các word có độ dài lớn hơn 3 (từ quá ngắn có thể là noise)
                len(word) <= 20): # lấy các word có độ dài nhỏ hơn 15 (từ quá dài có thể là noise))

                # đưa từ về dạng gốc
                lemmatized_word = self.lemmatizer.lemmatize(word)
                processed_words.append(lemmatized_word)

        return processed_words

    def preprocess_email(self, subject, message, max_tokens=50):
        """
        tiền xử lý 1 email (1 hàng của dữ liệu): làm sạch (sử dụng hàm clean_text và hàm tokenize_and_lemmatize)
        subject: tiêu đề thư
        message: nội dung thư
        max_tokens: giới hạn số token để tránh quá nhiều noise
        """
        # kết hợp subject và message
        combined_text = str(subject) + ' ' + str(message)

        # làm sạch văn bản
        cleaned_text = self.clean_text(combined_text)

        # Tokenize và lemmatize
        tokens = self.tokenize_and_lemmatize(cleaned_text)

        # kiểm tra số token
        if len(tokens) > max_tokens:
            tokens = tokens[:max_tokens]

        return tokens


    def preprocess_data(self, df, remove_duplicate=True):
        """
        Tiền xử lý 1 dataset(train/val)
        df: dataframe, đảm bảo tồn tại 3 cột subject, message, spam/ham (label)
        """
        # Thay thế giá trị NaN bằng chuỗi rỗng
        df['Subject'] = df['Subject'].fillna('')
        df['Message'] = df['Message'].fillna('')

        if remove_duplicate:
          # Kiểm tra dữ liệu trùng lặp
          duplicates = df.duplicated(subset=['Subject', 'Message'])
          print(f"Số lượng email trùng lặp: {duplicates.sum()}")

          # Loại bỏ trùng lặp nếu có
          df = df.drop_duplicates(subset=['Subject', 'Message']).reset_index(drop=True)
          print(f"Số lượng email sau khi loại bỏ trùng lặp: {len(df)}")

        # Tiền xử lý từng email
        processed_emails = []

        for idx, row in df.iterrows():
            tokens = self.preprocess_email(row['Subject'], row['Message'], max_tokens=50)

            processed_emails.append({
                'tokens': tokens,
                'label': row['Spam/Ham'],
                'original_subject': row['Subject'],
                'original_message': row['Message']
            })
        return processed_emails

## Giải thích các công thức và thành phần trong mô hình Naive Bayes

### 1. Xác suất tiên nghiệm của lớp (Prior Probability)
- Đây là xác suất xuất hiện của mỗi nhãn/lớp trước khi nhìn vào dữ liệu đầu vào.
- Công thức:  
  **P(class) = Số lượng tài liệu thuộc lớp đó / Tổng số tài liệu**

### 2. Xác suất có điều kiện của từ cho trước lớp (Likelihood)
- Với từng từ trong văn bản, tính xác suất nó xuất hiện trong một lớp cụ thể.
- Có 2 cách ước lượng:
  - **MLE (Maximum Likelihood Estimation):**  
    P(word | class) = Số lần từ xuất hiện trong lớp / Tổng số từ trong lớp
  - **MAP (Maximum A Posteriori Estimation):**  
    P(word | class) = (count + α) / (total + α * vocab_size)  
    (α là tham số Laplace smoothing, thường = 1)

### 3. Dự đoán nhãn bằng xác suất hậu nghiệm (Posterior Probability)
- Tính xác suất hậu nghiệm của từng lớp với một văn bản mới, sau đó chọn lớp có xác suất cao nhất.
- Dùng công thức Bayes:  
  **P(class | tokens) ∝ P(class) × Π P(word | class)**
- Vì tích của nhiều số nhỏ dễ gây tràn số, ta dùng log:
  **log P(class | tokens) = log P(class) + Σ log P(word | class)**

### 4. Xử lý từ không thấy trong tập huấn luyện
- Với MLE, xác suất có thể bằng 0 nếu từ chưa từng xuất hiện ⇒ log(0) bị lỗi.
- Giải pháp: dùng smoothing nhỏ như log(1e-10).
- Với MAP, Laplace smoothing giúp đảm bảo mọi từ đều có xác suất > 0.



In [24]:
class NaiveBayesClassifier:
    """
    Naive Bayes Classifier với MLE và MAP estimation
    """

    def __init__(self, alpha=1.0, estimation_method='MAP', min_word_freq=2):
        """
        alpha: tham số Laplace smoothing cho MAP
        method: 'MLE' hoặc 'MAP'
        min_word_freq: Tần số cần xuất hiện tối thiểu của 1 token trong 1 email để từ đó được giữ lại
        """
        self.alpha = alpha
        self.estimation_method = estimation_method
        self.min_word_freq = min_word_freq

        # vocabulary là các unique tokens trong dataset
        self.vocabulary = set()
        # đếm số lượng emails theo từng class, ví dụ class_counts["spam"] = 101, nghĩa là có 101 emails spam trong dataset
        self.class_counts = {}
        # đếm frequency của từng từ trong class "spam" và "ham", ví dụ, word_counts["spam"]["hello"] = 51, nghĩa là có 51 email có từ hello đc đánh dấu là spam
        self.word_counts = {}
        # tính class priors P(class), ví dụ P["spam"] = số lượng email spam / tổng email
        self.class_priors = {}
        # là P(word|class), ví dụ P["spam"]["hello"] = 0.2
        self.word_probs = {}
        # số lượng emails
        self.total_docs = 0
        # số lượng unique tokens trong dataset
        self.vocab_size = 0

    def mle_estimation(self):
        """Maximum Likelihood Estimation"""

        self.word_probs = defaultdict(dict)

        for class_name in self.class_counts:
            total_words_in_class = sum(self.word_counts[class_name].values())

            for word in self.vocabulary:
                word_count = self.word_counts[class_name].get(word, 0)
                # MLE: P(word|class) = count(word, class) / count(class)
                self.word_probs[class_name][word] = word_count / total_words_in_class if total_words_in_class > 0 else 0

    def map_estimation(self):
        """Maximum A Posteriori Estimation với Laplace smoothing"""

        self.word_probs = defaultdict(dict)

        for class_name in self.class_counts:
            total_words_in_class = sum(self.word_counts[class_name].values())

            for word in self.vocabulary:
                word_count = self.word_counts[class_name].get(word, 0)
                # MAP với Laplace smoothing: P(word|class) = (count + alpha) / (total + alpha * vocab_size)
                self.word_probs[class_name][word] = (word_count + self.alpha) / (total_words_in_class + self.alpha * len(self.vocabulary))

    def fit(self, emails):
        """Huấn luyện mô hình với MLE hoặc MAP"""

        # Reset parameters
        self.class_counts = defaultdict(int)
        self.word_counts = defaultdict(lambda: defaultdict(int))

        # vocabulary là unique tokens trong dataset
        self.vocabulary = set()

        # Đếm số lượng emails theo từng class và xây dựng vocabulary
        for email in emails:
            tokens = email['tokens']
            label = email['label']

            self.class_counts[label] += 1
            self.vocabulary.update(tokens)

            # Đếm frequency của từng từ trong class
            for token in tokens:
                self.word_counts[label][token] += 1

        self.total_docs = len(emails)
        self.vocab_size = len(self.vocabulary)

        # Tính class priors P(class)
        for class_name in self.class_counts:
            self.class_priors[class_name] = self.class_counts[class_name] / self.total_docs

        # Parameter estimation
        if self.estimation_method == 'MLE':
            self.mle_estimation()
        else:  # MAP
            self.map_estimation()

        # Thống kê
        print(f"Số lượng vocabulary: {self.vocab_size}")
        print(f"Class distribution: {dict(self.class_counts)}")
        print(f"Class priors: {dict(self.class_priors)}")

    def predict_proba(self, tokens):
        """Tính log probabilities cho từng class"""
        scores = {}

        # Chỉ xét các từ có trong selected features
        relevant_tokens = [token for token in tokens if token in self.vocabulary]

        for class_name in self.class_counts:
            # Bắt đầu với log prior: log P(class)
            log_prob = math.log(self.class_priors[class_name])

            # Tính log likelihood: sum(log P(word|class))
            for token in relevant_tokens:
                if token in self.word_probs[class_name]:
                    word_prob = self.word_probs[class_name][token]

                    # Tránh log(0)
                    if word_prob > 0:
                        log_prob += math.log(word_prob)
                    else:
                        # Xử lý từ không xuất hiện (chỉ với MLE)
                        if self.estimation_method == 'MLE':
                            log_prob += math.log(1e-10)  # Smoothing nhỏ

            scores[class_name] = log_prob

        return scores

    def predict_one(self, tokens):
        """predict cho một email"""
        scores = self.predict_proba(tokens)
        return max(scores, key=scores.get)

    def predict_many(self, emails_tokens):
        """predict cho nhiều emails"""
        predictions = []

        for i, tokens in enumerate(emails_tokens):
            pred = self.predict_one(tokens)
            predictions.append(pred)

        return predictions

def eval(model, eval_emails):
    test_tokens = [email['tokens'] for email in eval_emails]
    true_labels = [email['label'] for email in eval_emails]

    # predict trên toàn bộ data eval
    predictions = model.predict_many(test_tokens)

    # tính accuracy
    correct = sum(1 for true, pred in zip(true_labels, predictions) if true == pred)
    accuracy = correct / len(true_labels)

    # tính F1 cho từng class
    metrics = {}
    f1_scores = []

    for class_name in sorted(set(true_labels)):
        tp = sum(1 for true, pred in zip(true_labels, predictions) if true == pred == class_name)
        fp = sum(1 for true, pred in zip(true_labels, predictions) if true != class_name and pred == class_name)
        fn = sum(1 for true, pred in zip(true_labels, predictions) if true == class_name and pred != class_name)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        metrics[class_name] = {'precision': precision, 'recall': recall, 'f1': f1}
        f1_scores.append(f1)

    # f1 tổng
    f1 = sum(f1_scores) / len(f1_scores)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Số lượng dự đoán đúng: {correct}/{len(true_labels)}")
    print(f"F1-score (Average): {f1:.4f}")

    for class_name, m in metrics.items():
        print(f"{class_name}: Precision: {m['precision']:.4f}, Recall: {m['recall']:.4f}, F1-score: {m['f1']:.4f}")

## TRAINING

In [25]:
# đọc dữ liệu training
df_train = pd.read_csv('/content/sample_data/train.csv')
df_train.head()

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,split
0,0,0,christmas tree farm pictures,NaN,ham,0.038415
1,1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,0.696509
2,2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,0.587792
3,3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,-0.055438
4,5,5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",ham,-0.419658


In [ ]:
# Training
# Tiền xử lí dữ liệu cho dữ liệu train
preprocessor = TextPreprocessor()
df_train = preprocessor.preprocess_data(df_train, remove_duplicate=True)

# MLE
print("Training với mô hình thống kê MLE")
model_mle = NaiveBayesClassifier(
    estimation_method='MLE',
    alpha=1.0,
    min_word_freq=3
)
model_mle.fit(df_train)

# MAP
print("Training với mô hình thống kê MAP")
model_map = NaiveBayesClassifier(
    estimation_method='MAP',
    min_word_freq=3
)
model_map.fit(df_train)

## CODE BLOCK CHỨC NĂNG 1: Người dùng nhập vào một email bất kì (gồm tiêu đề và nội dung).

In [18]:
def predict_custom_email(subject, message, model, preprocessor):
    preprocessor = TextPreprocessor()
    processed_tokens = preprocessor.preprocess_email(subject, message)
    prediction = model.predict_one(processed_tokens)
    return prediction

# yêu cầu nhập thông tin
print("Nhập thông tin email bạn muốn kiểm tra.")
user_subject = input("Tiêu đề (Subject): ")
user_message = input("Nội dung (Message): ")

if user_subject.strip() or user_message.strip():
    # Kết quả của mô hình MLE
    prediction_mle = predict_custom_email(user_subject, user_message, model_mle, preprocessor)
    print(f"Kết quả dự đoán từ mô hình MLE: {prediction_mle.upper()}")

    # Kết quả của mô hình MAP
    prediction_map = predict_custom_email(user_subject, user_message, model_map, preprocessor)
    print(f"Kết quả dự đoán từ mô hình MAP: {prediction_map.upper()}")


Nhập thông tin email bạn muốn kiểm tra.
Tiêu đề (Subject): how are you?
Nội dung (Message): hi, what's your name?
Kết quả dự đoán từ mô hình MLE: SPAM
Kết quả dự đoán từ mô hình MAP: SPAM


## CODE BLOCK CHỨC NĂNG 2: Eval trên tập dữ liệu tương tự "val.csv"

Thầy thay đường dẫn val_path đến file csv trong code block sau


In [19]:
# evaluate
# đường dẫn tới file csv cần eval
val_path = '/content/sample_data/val.csv'
df_val = pd.read_csv(val_path)
# tiền xử lí dữ liệu (do eval nên không loại bỏ dữ liệu trùng lặp)
preprocessor = TextPreprocessor()
df_val = preprocessor.preprocess_data(df_val, remove_duplicate=False)
# eval trên mô hình MAP
print("Eval trên mô hình MAP")
eval(model_map, df_val)
# eval trên mô hình MLE
print("Eval trên mô hình MLE")
eval(model_mle, df_val)

Eval trên mô hình MAP
Accuracy: 0.9812
Số lượng dự đoán đúng: 3026/3084
F1-score (Average): 0.9812
ham: Precision: 0.9747, Recall: 0.9875, F1-score: 0.9811
spam: Precision: 0.9877, Recall: 0.9750, F1-score: 0.9813
Eval trên mô hình MLE
Accuracy: 0.9822
Số lượng dự đoán đúng: 3029/3084
F1-score (Average): 0.9822
ham: Precision: 0.9785, Recall: 0.9855, F1-score: 0.9820
spam: Precision: 0.9858, Recall: 0.9789, F1-score: 0.9823
